# Introduction
This Colab explore the possibilities for Federated Machine Learning accross Data Vaults. 

# Preparation

In [5]:
#@title Install Libraries
!pip install -q tensorflow==2.5.1
!pip install -q tensorflow-federated==0.19.0
!pip install -q nest_asyncio==1.4.3
print("pip installations finished. consider restarting kernel")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-client 7.0.2 requires nest-asyncio>=1.5, but you have nest-asyncio 1.4.3 which is incompatible.
pip installations finished. consider restarting kernel


In [6]:
!pip list

Package                           Version
--------------------------------- -------------------
absl-py                           0.13.0
aiohttp                           3.7.4.post0
ansiwrap                          0.8.4
anyio                             3.3.0
apache-beam                       2.32.0
appdirs                           1.4.4
argcomplete                       1.12.3
argon2-cffi                       20.1.0
arrow                             1.1.1
asn1crypto                        1.4.0
astunparse                        1.6.3
async-generator                   1.10
async-timeout                     3.0.1
attrs                             19.3.0
avro-python3                      1.9.2.1
backcall                          0.2.0
backports.entry-points-selectable 1.1.0
backports.functools-lru-cache     1.6.4
beatrix-jupyterlab                0.9.1
binaryornot                       0.4.4
black                             21.8b0
bleach                            4.1.0
blinker    

In [7]:
#@title Import Libraries
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np
import nest_asyncio
import grpc
import sys
import os

nest_asyncio.apply()
tf.get_logger().setLevel('ERROR')
print("Running TFF version", tff.__version__)
print("And TF version", tf.__version__)
!python --version

2021-09-21 20:25:26.479701: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Running TFF version 0.19.0
And TF version 2.5.1
Python 3.7.10


In [8]:
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        print("Its a colab")
        from google.colab import auth as google_auth
        google_auth.authenticate_user()
    else: 
        print("Its something else")
else:
    print("Its a managed notebook")

Its a managed notebook


# TFF Hello World


In [9]:
@tff.tf_computation(tf.int64)
def make_data(n):
  return tf.data.Dataset.range(n)

@tff.tf_computation(tff.SequenceType(tf.int64))
def sum_data(dataset):
  return dataset.reduce(np.int64(0), lambda x, y: x + y)

@tff.federated_computation(tff.FederatedType(tf.int64, tff.CLIENTS))
def compute_local_sum(federated_n):
 dataset   = tff.federated_map(make_data, federated_n)
 local_sum = tff.federated_map(sum_data, dataset)
 return local_sum

@tff.federated_computation(tff.FederatedType(tf.int64, tff.CLIENTS))
def compute_federated_sum(federated_n):
 return tff.federated_sum(compute_local_sum(federated_n))

print("sum_data:"     , compute_local_sum.type_signature)
print("compute_local_sum:"     , compute_local_sum.type_signature)
print("compute_federated_sum:" , compute_federated_sum.type_signature)

sum_data: ({int64}@CLIENTS -> {int64}@CLIENTS)
compute_local_sum: ({int64}@CLIENTS -> {int64}@CLIENTS)
compute_federated_sum: ({int64}@CLIENTS -> int64@SERVER)


In [10]:
service_url = !gcloud run services describe tff-executor-federation --region=europe-west3 --format 'value(status.url)' 
service_url = service_url[0].replace("https://","")
print("URL of the service is: " + service_url)

URL of the service is: tff-executor-federation-lrkxgk4hia-ey.a.run.app


In [11]:
# Get a new token. Run this again when the token expires
token = !gcloud auth print-identity-token

In [ ]:
nest_asyncio.apply()

# Set the credentials and channel 
scc = grpc.ssl_channel_credentials()
tok = grpc.access_token_call_credentials(token[0])
ccc = grpc.composite_channel_credentials(scc, tok)

#channels = [grpc.secure_channel(service_url + ":443",credentials=ccc)]
channels = [grpc.insecure_channel('34.107.126.200:8000')]
tff.backends.native.set_remote_execution_context(channels, default_num_clients=1)

compute_federated_sum([7])